---

#  Examples using Package `pdpolygonapi`

---


In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%matplotlib inline
#%matplotlib qt
import pandas as pd
import mplfinance as mpf
#import yfinance as yf

In [3]:
# import requests
# import datetime
# # import numpy as np
# # import warnings
# import os
# APIKEY=os.environ.get('POLYGON_API')

In [1]:
import pdpolygonapi

In [2]:
dir(pdpolygonapi)

['PolygonApi',
 '_PolygonApiBase',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_pdpolygonapi_base',
 'pdpolygonapi']

In [1]:
from pdpolygonapi import PolygonApi

In [3]:
dir(PolygonApi)

['OptionsChain',
 '_OHLCV_COLMAP',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_input_to_datetime',
 '_input_to_mstimestamp',
 '_json_response_to_ohlcvdf',
 '_req_get_json',
 'fetch_ohlcvdf',
 'fetch_options_chain',
 'fetch_quotes']

In [3]:
polygon = pdpolygonapi.PolygonApi(envkey='POLYGON_API')

In [6]:
start_date = '2022-03-01'
end_date   = '2023-08-01'
opchain    = polygon.fetch_options_chain('SPY',start_date,end_date)

req=
 https://api.polygon.io/v3/reference/options/contracts?underlying_ticker=SPY&expired=true&expiration_date.gte=2022-03-01&expiration_date.lte=2023-08-01&limit=1000&apiKey=***
.......................................................................................

In [7]:
opchain.underlying
len(opchain.tickers)
len(opchain.expirations)
len(opchain.strikes)
for item in dir(opchain):
    if item[0] == '_': continue
    print(item)

opchain.expirations.head(3)
opchain.expirations.tail(3)

'SPY'

87920

290

290

expirations
get_strikes_by_expiration
strikes
tickers
underlying


,Expiration
2022-03-02,2022-03-02
2022-03-04,2022-03-04
2022-03-07,2022-03-07


,Expiration
2023-07-28,2023-07-28
2023-07-31,2023-07-31
2023-08-01,2023-08-01


In [8]:
sdt = (pd.Timestamp(start_date) - pd.Timedelta(days=65))
spy = polygon.fetch_ohlcvdf('SPY',sdt,end_date)

dtm= 2023-08-01 23:59:59.099999
dtm= 2021-12-26 00:00:00
req=
 https://api.polygon.io/v2/aggs/ticker/SPY/range/1/day/1640476800000/1690948799099?adjusted=true&sort=asc&limit=50000&apiKey=***


In [9]:
spy

,Open,High,Low,Close,Volume
SPY,,,,,
2021-12-27,472.06,477.31,472.0100,477.26,56808619.0
2021-12-28,477.72,478.81,476.0600,476.87,46974585.0
2021-12-29,476.98,478.56,475.9200,477.48,54091464.0
2021-12-30,477.93,479.00,475.6700,476.16,55329041.0
2021-12-31,475.64,476.86,474.6700,474.96,64917431.0
...,...,...,...,...,...
2023-07-26,454.47,456.99,453.3800,455.51,70897885.0
2023-07-27,459.02,459.44,451.5500,452.49,92090375.0
2023-07-28,455.88,457.78,452.4918,456.92,80001790.0


In [10]:
xdt = opchain.expirations.index[0]
xdt
xdt + datetime.timedelta(days=-60)

Timestamp('2022-03-02 00:00:00')

Timestamp('2022-01-01 00:00:00')

In [12]:
opchain.strikes[str(xdt.date())]

,Strike
Strike,
285.0,285.0
290.0,290.0
295.0,295.0
300.0,300.0
305.0,305.0
...,...
520.0,520.0
525.0,525.0
530.0,530.0


In [13]:
strikes = opchain.strikes[str(xdt.date())]
strikes.loc[450:].iloc[3][0]

453.0

In [16]:
jj=0
for xdt in opchain.expirations.index:
    print('\n\n===',xdt,'===')
    sdt = xdt - pd.Timedelta(days=60)
    edt = xdt + pd.Timedelta(days=1)
    pxs = spy.loc[sdt:edt]
    hi = max(pxs.High)
    lo = min(pxs.Low)
    strikes = opchain.strikes[str(xdt.date())]
    shi = strikes.loc[hi:].iloc[3][0]
    #print('hi=',hi,'shi=',shi)
    slo = strikes.loc[:lo].iloc[-4][0]
    #print('lo=',lo,'slo=',slo)
    relevant_stikes = strikes[slo:shi]

    tickers = opchain.tickers.loc[(str(xdt.date()),slo):(str(xdt.date()),shi)].Ticker.values
    len(tickers)
    midtkr = tickers[int(len(tickers)/2)]
    print(midtkr)
    jj += 1
    if jj > 1: break



=== 2022-03-02 00:00:00 ===


154

O:SPY220302P00445000


=== 2022-03-04 00:00:00 ===


154

O:SPY220304P00445000


In [76]:
str_date = '2023-08-09'
ts1 = str(int(pd.Timestamp(str_date+' 09:30',tz='US/Eastern').tz_convert('UTC').timestamp()*(10**9)))

In [78]:
int(ts1)

1691587800000000000

In [86]:
ts = pd.Timestamp(int(ts1))

In [88]:
ts
str(ts)

Timestamp('2023-08-09 13:30:00')

'2023-08-09 13:30:00'

In [6]:
ticker   = 'O:SPY230818P00450000'
str_date = '2023-08-09'
qdf = polygon.fetch_quotes(ticker,str_date)

Requesting quote data for "O:SPY230818P00450000"
 from 2023-08-09 13:30:00  to  2023-08-09 20:00:00 UTC
req=
 https://api.polygon.io/v3/quotes/O:SPY230818P00450000?timestamp.gte=1691587800000000000&timestamp.lte=1691611200000000000&limit=50000&apiKey=***
response status: OK
received 50000 quotes so far ...
getting next_url ... response status: OK  received 100000 quotes so far ...
getting next_url ... response status: OK  received 150000 quotes so far ...
getting next_url ... response status: OK  received 200000 quotes so far ...
getting next_url ... response status: OK  received 250000 quotes so far ...
getting next_url ... response status: OK  received 296889 quotes.
resampling to 1S intervals ...
returning 22055 quotes.


In [7]:
qdf.to_csv('temp.csv.gz')
qdf.to_csv('temp.csv')

In [8]:
%ls -l temp*

-rw-r--r-- 1 dino dino 1366892 Aug 11 15:38 temp.csv
-rw-r--r-- 1 dino dino  316895 Aug 11 15:38 temp.csv.gz


In [9]:
!head temp.csv

Timestamp,Ask,AsizeA,AsizeM,AsizeH,AsizeL,Bid,BsizeA,BsizeM,BsizeH,BsizeL,Count
2023-08-09 09:30:00,3.91,78,40,246,5,3.89,46,50,128,5,19
2023-08-09 09:30:01,3.9,75,59,109,40,3.88,68,64,109,1,8
2023-08-09 09:30:02,3.89,103,110,215,1,3.87,98,110,129,19,11
2023-08-09 09:30:03,3.9,110,109,259,1,3.88,116,129,129,45,15
2023-08-09 09:30:04,3.9,115,110,236,1,3.88,110,129,129,1,26
2023-08-09 09:30:05,3.89,41,2,190,1,3.88,102,129,129,5,9
2023-08-09 09:30:06,3.91,75,59,202,1,3.9,97,116,129,1,14
2023-08-09 09:30:07,3.92,153,200,311,1,3.9,130,129,349,1,19
2023-08-09 09:30:08,3.91,146,137,311,2,3.89,146,110,330,19,21


In [10]:
qdf2 = pd.read_csv('temp.csv.gz',index_col='Timestamp',parse_dates=True)

In [11]:
 qdf.head(3)
qdf2.head(3)
 qdf.tail(3)
qdf2.tail(3)

,Ask,AsizeA,AsizeM,AsizeH,AsizeL,Bid,BsizeA,BsizeM,BsizeH,BsizeL,Count
Timestamp,,,,,,,,,,,
2023-08-09 09:30:00,3.91,78,40,246,5,3.89,46,50,128,5,19
2023-08-09 09:30:01,3.90,75,59,109,40,3.88,68,64,109,1,8
2023-08-09 09:30:02,3.89,103,110,215,1,3.87,98,110,129,19,11


,Ask,AsizeA,AsizeM,AsizeH,AsizeL,Bid,BsizeA,BsizeM,BsizeH,BsizeL,Count
Timestamp,,,,,,,,,,,
2023-08-09 09:30:00,3.91,78,40,246,5,3.89,46,50,128,5,19
2023-08-09 09:30:01,3.90,75,59,109,40,3.88,68,64,109,1,8
2023-08-09 09:30:02,3.89,103,110,215,1,3.87,98,110,129,19,11


,Ask,AsizeA,AsizeM,AsizeH,AsizeL,Bid,BsizeA,BsizeM,BsizeH,BsizeL,Count
Timestamp,,,,,,,,,,,
2023-08-09 15:59:57,5.86,37,47,73,1,5.83,200,247,294,30,36
2023-08-09 15:59:58,5.88,42,47,169,1,5.85,112,98,246,1,26
2023-08-09 15:59:59,5.90,97,62,248,46,5.87,128,62,327,1,34


,Ask,AsizeA,AsizeM,AsizeH,AsizeL,Bid,BsizeA,BsizeM,BsizeH,BsizeL,Count
Timestamp,,,,,,,,,,,
2023-08-09 15:59:57,5.86,37,47,73,1,5.83,200,247,294,30,36
2023-08-09 15:59:58,5.88,42,47,169,1,5.85,112,98,246,1,26
2023-08-09 15:59:59,5.90,97,62,248,46,5.87,128,62,327,1,34


In [12]:
all(qdf.columns == qdf2.columns)
qdf.index.name == qdf2.index.name

len(qdf) == len(qdf2)
all(qdf.index == qdf2.index)

type(qdf.index)
type(qdf2.index)

all(qdf == qdf2)

True

True

True

True

pandas.core.indexes.datetimes.DatetimeIndex

pandas.core.indexes.datetimes.DatetimeIndex

True

In [ ]:
STOP HERE STOP HERE STOP HERE

In [ ]:
print('''

     Keep this for now.  Not sure if still useful.

''')

def polygon_option_chain_to_df(results):
    d   = pd.json_normalize(results,sep=':').to_dict()
    tdf = pd.DataFrame(d)
    tdf.drop(['day:change','day:change_percent','details:exercise_style',
              'details:shares_per_contract','underlying_asset:ticker'],
             axis=1,errors='ignore',inplace=True)
    newcols = []
    for col in tdf.columns:
        newcols.append(col.split(':')[-1])
    tdf.columns = newcols
    column_mappings = {'ticker':'Ticker',
                        'open_interest':'OI',
                        'close':'Close',
                        'high':'High',
                        'last_updated':'Timestamp',
                        'low':'Low',
                        'open':'Open',
                        'previous_close':'PrevClose',
                        'volume':'Volume',
                        'vwap': 'VWAP',
                        'contract_type':'PC',
                        'expiration_date':'Expiration',
                        'strike_price':'Strike',
                       }
    tdf.rename(columns=column_mappings,inplace=True)
    tdf.set_index('Ticker',inplace=True)
    return tdf